<a href="https://colab.research.google.com/github/xanasa14/MLImplementations/blob/master/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chatterbot
!pip install chatterbot_corpus

     |████████████████████████████████| 71kB 6.3MB/s 
     |████████████████████████████████| 122kB 12.1MB/s 


In [10]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from chatterbot.trainers import ChatterBotCorpusTrainer

corpus_trainer = ChatterBotCorpusTrainer(my_bot)
corpus_trainer.train('chatterbot.corpus.english')


Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


In [6]:
my_bot = ChatBot(name='PyBot', read_only=True, logic_adapters=
                 ['chatterbot.logic.MathematicalEvaluation',
                  'chatterbot.logic.BestMatch'])

In [7]:
small_talk = ["hi there","hi","How are you?",
              'How do you do?','I\'m cool','fine, you?']
math_talk1 = ['Pythagorean theorem', 'squared plus b squared equals c squared']
math_talk2 = ['law of cosines', 'c**2 = a**2 + b**2 -2 * a * b * cos(gamma']

In [8]:
list_trainer = ListTrainer(my_bot)
for item in (small_talk, math_talk1, math_talk2):
  list_trainer.train(item)

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%


In [9]:
print(my_bot.get_response("hi"))
print(my_bot.get_response("How are you?"))

How are you?
How do you do?
